Load documents with IDs

In [ ]:
import json

with open('documents-with-idxs.json') as f_in:
    documents = json.load(f_in)

Load ground truth

In [ ]:
import pandas as pd

df_ground_truth = pd.read_csv("ground-truth-data.csv")
df_ground_truth = df_ground_truth[df_ground_truth['course'] == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient="records")

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-MiniLM-L6-cos-v1"
sentence_model = SentenceTransformer(model_name)

Index data in Elasticsearch-kibana

In [ ]:
from elasticsearch import Elasticsearch

# # With kibana
# es_client = Elasticsearch("https://172.18.0.2:9200/", basic_auth=["elastic", "e7QvGtIFflWimVkpt3iv"], verify_certs=False, ssl_show_warn=False)

# Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    question = doc['question']
    text = doc["text"]
    doc["question_text_vector"] = sentence_model.encode(question + ' ' + text)

    es_client.index(index=index_name, document=doc)

Retrieval

In [ ]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_text_vector_knn(query):
    question = query['question']
    course = query['course']

    q_v = sentence_model.encode(question)

    return elastic_search_knn('question_text_vector', q_v, course)

In [ ]:
search_results = question_text_vector_knn(dict(
    question='Which city is the capital of the Turkey',
    course='machine-learning-zoomcamp'
))

The RAG flow

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

tokenizer = AutoTokenizer.from_pretrained("cognitivecomputations/dolphin-2_6-phi-2", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "cognitivecomputations/dolphin-2_6-phi-2",
      torch_dtype="auto",
        trust_remote_code=True,) ## attn_implementation="flash_attention_2",

In [ ]:
def llm(prompt):
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device="cuda",)
    
    generation_args = {
        "max_new_tokens": 100,
        "return_full_text": False,
        "temperature": 0.5,
        "do_sample": True,
    }

    output = pipe([{"role": "user", "content": prompt}], **generation_args)

    return output[0]['generated_text']


def generate_question(query):
    search_results = question_text_vector_knn(query)
    prompt = build_prompt(query['question'], search_results)

    response = llm(prompt)

    return response

In [ ]:
doc_idx = {doc['id']: doc for doc in documents}

In [ ]:
answers = {}

for i, rec in enumerate(tqdm(ground_truth)):
    if i in answers:
        continue

    answer_llm = generate_question(rec)
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    answers[i] = {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }